In [1]:
import json
import random
from glob import glob
from tqdm import tqdm
from transformers import AutoTokenizer

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
synthetic = []

for f in glob('/home/husein/ssd3/instructions/synthetic-*.jsonl'):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            
            if l['rejected_ins'] or l['rejected_output']:
                continue
                
            if l['instruction_ms'] is None:
                continue
            
            if l['output_ms'] is None:
                continue
                
            synthetic.append({
                'prompt_input': None,
                'input': l['instruction_ms'].strip(),
                'output': l['output_ms'].strip(),
            })
                
len(synthetic)

536851

In [3]:
general_qa = []
for f in glob('/home/husein/ssd3/chatgpt4-malaysian-general-qa/*.jsonl'):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            q = l.get('question_ms', l['question']).strip()
            if not len(q):
                continue
            a = l.get('answer_ms', l['answer'])
            if a is None:
                continue
            a = a.strip()
            if not len(a):
                continue
                
            d = {
                'prompt_input': None,
                'input': q,
                'output': a,
            }
            general_qa.append(d)
            
len(general_qa)

68234

In [4]:
chatgpt4_code = []
with open('/home/husein/ssd3/instructions/synthetic-codealpaca-v1-chatgpt4.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        q = l.get('instruction_ms')
        if q is None:
            q = l['instruction']
        q = q.strip()
        if not len(q):
            continue
        a = l.get('output_ms', l['output'])
        if a is None:
            continue
        a = a.strip()
        if not len(a):
            continue

        d = {
            'prompt_input': None,
            'input': q,
            'output': a,
        }
        chatgpt4_code.append(d)
            
len(chatgpt4_code)

43337

In [5]:
random_instructions = {
    0: synthetic,
    1: synthetic,
    2: synthetic,
    3: general_qa,
    4: chatgpt4_code
}

In [6]:
random.randint(2, 4)

4

In [7]:
roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

In [8]:
random_combine = []
for i in tqdm(range(100000)):
    combine = []
    for _ in range(random.randint(2, 4)):
        index = random.randint(0, 4)
        choice = random.choice(random_instructions[index])
        s = f"<manusia>: {choice['input']}"
        combine.append(s)
        s = f"<bot>: {choice['output']}"
        combine.append(s)
    data = '\n'.join(combine).strip()
    a = {
        'prompt_input': None,
        'input': data,
        'output': None
    }
    random_combine.append(a)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 104952.14it/s]


In [9]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'model bahasa AI',
    'mempunyai kepercayaan atau pendapat peribadi',
    'tidak mempunyai pendapat peribadi',
    'tidak mempunyai kepercayaan',
    'tidak mempunyai falsafah peribadi',
    'tidak mempunyai pengalaman peribadi',
    'tidak mempunyai pendapat atau pengalaman peribadi',
    'tidak mempunyai maklumat terkini',
    'tidak mempunyai emosi peribadi',
    'tidak mempunyai keutamaan',
    'saya tidak mempunyai akses',
    'tidak mempunyai pengalaman',
    'saya tidak mempunyai keupayaan',
    'tidak mempunyai keupayaan',
    'tidak mempunyai hubungan',
    'tidak mempunyai maklumat',
    'Saya tidak mempunyai',
    'Saya tidak pernah',
]

In [10]:
glob('/home/husein/ssd3/gov.my/ultrachat-*.jsonl')

['/home/husein/ssd3/gov.my/ultrachat-jurnaldbp.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-jurnaldbp-malay.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-maktabahalbakri.com.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-gov.my.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-crossref-melayu.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-lom-agc.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-astroawani-malay.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-textbooks.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-crossref-melayu-malay.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-epenerbitan-malay.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-hansard-malay.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-ms-wikipedia.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-glaive_coder_raw_text.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-muftiwp.gov.my.texts.jsonl']

In [ ]:
data_instructions = []

count, count1 = 0, 0
for file in glob('/home/husein/ssd3/gov.my/ultrachat-*.jsonl'):
    with open(file) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            
#             if l[1]['content'] is None:
#                 print(l)
#                 continue
                
            context = l[0]['content']
                
            l = l[1:]
            inputs = []
            for no, r in enumerate(l):
                
                if no < (len(l) - 1):
                    next_text = l[no + 1].get('content_ms') or ''
                else:
                    next_text = ''
                    
                current_text = r.get('content_ms') or ''
                previous_text = l[no - 1].get('content_ms') or ''
                
                # bad pairs
                if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                    continue
                if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                    continue
                
                # bad pairs
                if r['role'] == 'user' and r['content_ms'][:20].lower() == next_text[:20].lower():
                    # print(no, r, r['content_ms'][:20].lower(), next_text[:20].lower())
                    continue
                if r['role'] == 'assistant' and r['content_ms'][:20].lower() == l[no - 1]['content_ms'][:20].lower():
                    # print(no, r, r['content_ms'][:20].lower(), l[no - 1]['content_ms'][:20].lower())
                    continue
                    
                # remove alignments    
                if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                    continue
                if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                    continue

                role = roles[r['role']]
                
                if no == 0 and ('crossref-melayu' in file or random.random() > 0.7):
                    s = f"{role}: {context}\n\n{r['content_ms']}"
                    count += 1
                else:
                    s = f"{role}: {r['content_ms']}"
                    
                inputs.append(s)
                count1 += 1

            if len(inputs) % 2 != 0:
                inputs = inputs[:-1]
                
            if not len(inputs):
                continue
            
            if random.random() > 0.4:
                continue
            
            index = random.randint(0, 4)
            choice = random.choice(random_instructions[index])
            manusia = f"<manusia>: {choice['input']}"
            bot = f"<bot>: {choice['output']}"
            
            if random.random() > 0.5:
                inputs.extend([manusia, bot])
            else:
                inputs = [manusia, bot] + inputs

            data = '\n'.join(inputs).strip()
            
            if not len(data):
                continue
                
            a = {
                'prompt_input': None,
                'input': data,
                'output': None
            }
            data_instructions.append(a)
        
        
len(data_instructions)

1734it [00:01, 1010.87it/s]
6440it [00:03, 1867.80it/s]
3350it [00:01, 1806.91it/s]
10128it [00:05, 2006.19it/s]
1296it [00:01, 1229.47it/s]
8044it [00:05, 1600.66it/s]
60198it [00:16, 3680.14it/s]
7037it [00:04, 1440.99it/s]

In [ ]:
from tqdm import tqdm

data_instructions = []

count, count1 = 0, 0
for file in files:
    with open(file) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            
#             if l[1]['content'] is None:
#                 print(l)
#                 continue
            
            if 'glaive_coder_raw_text' in file:
                code_instruct = True
            else:
                code_instruct = False
                
            context = l[0]['content']
            
            if not code_instruct:
                l = l[1:]
                
            inputs = []
            for no, r in enumerate(l):
                
                if no < (len(l) - 1):
                    if l[no + 1]['role'] == 'user':
                        next_text = l[no + 1].get('content_ms') or l[no + 1].get('content') or ''
                    else:
                        next_text = l[no + 1].get('content_ms') or ''
                else:
                    next_text = ''
                
                if r['role'] == 'user':
                    current_text = r.get('content_ms') or r.get('content') or ''
                else:
                    current_text = r.get('content_ms') or ''
                    
                if l[no - 1]['role'] == 'user':
                    previous_text = l[no - 1].get('content_ms') or l[no - 1].get('content') or ''
                else:
                    previous_text = l[no - 1].get('content_ms') or ''
                
                # bad pairs
                if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                    # print('a', l, current_text, next_text, '\n')
                    continue
                if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                    continue
                
                # bad pairs
                if not code_instruct and r['role'] == 'user' and current_text[:20].lower() == next_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), next_text[:20].lower())
                    continue
                if not code_instruct and r['role'] == 'assistant' and current_text[:20].lower() == previous_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), previous_text[:20].lower())
                    continue
                    
                # remove alignments    
                if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                    continue
                if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                    continue

                role = roles[r['role']]
                
                if no == 0 and not code_instruct and ('crossref-melayu' in file or random.random() > 0.7):
                    s = f"{role}: {context}\n\n{current_text}"
                    count += 1
                else:
                    s = f"{role}: {current_text}"
                    
                inputs.append(s)
                count1 += 1

            if len(inputs) % 2 != 0:
                inputs = inputs[:-1]
                
            if not len(inputs):
                continue
            
            if random.random() > 0.4:
                continue
            
            index = random.randint(0, 4)
            choice = random.choice(random_instructions[index])
            manusia = f"<manusia>: {choice['input']}"
            bot = f"<bot>: {choice['output']}"
            
            if random.random() > 0.5:
                inputs.extend([manusia, bot])
            else:
                inputs = [manusia, bot] + inputs

            data = '\n'.join(inputs).strip()
            
            if not len(data):
                continue
                
            a = {
                'prompt_input': None,
                'input': data,
                'output': None
            }
            data_instructions.append(a)
        
        
len(data_instructions)

In [ ]:
data_instructions[0]

In [ ]:
combine_all = data_instructions + random_combine
len(combine_all)

In [ ]:
from tqdm import tqdm

for i in tqdm(range(len(combine_all))):
    generate_and_tokenize_prompt(combine_all[i])

In [ ]:
with open('prepared-combine-multiple-chats.jsonl', 'w') as fopen:
    for l in combine_all:
        fopen.write(f'{json.dumps(l)}\n')

In [ ]:
!wc -l prepared-combine-multiple-chats.jsonl

In [ ]:
!ls -lh prepared-combine-multiple-chats.jsonl

In [ ]:
!ls -lh prepared-malaysian-ultrachat.jsonl